In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.driver.memory", '4g')
    .config("spark.executor.memory", '8g')
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.sql.parquet.enableVectorizedReader","false")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .getOrCreate()
)

22/09/08 14:20:19 WARN Utils: Your hostname, dash_surface resolves to a loopback address: 127.0.1.1; using 172.26.20.143 instead (on interface eth0)
22/09/08 14:20:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/08 14:20:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
sdf_transactions = spark.read.parquet('../data/tables/transactions_20210228_20210827_snapshot')

In [3]:
sdf_transactions.show(10, truncate=False)

+-------+------------+------------------+------------------------------------+--------------+
|user_id|merchant_abn|dollar_value      |order_id                            |order_datetime|
+-------+------------+------------------+------------------------------------+--------------+
|18478  |62191208634 |63.255848959735246|949a63c8-29f7-4ab0-ada4-99ac50a88952|2021-08-20    |
|2      |15549624934 |130.3505283105634 |6a84c3cf-612a-4574-835b-144a47353eff|2021-08-20    |
|18479  |64403598239 |120.15860593212783|b10dcc33-e53f-4254-863c-de5266810cbc|2021-08-20    |
|3      |60956456424 |136.6785200286976 |0f09c5a5-784e-4477-b049-8ee4dd069b7b|2021-08-20    |
|18479  |94493496784 |72.96316578355305 |f6c78c1a-4600-4c5f-8e97-6e9eb534b586|2021-08-20    |
|3      |76819856970 |448.529684285612  |5ace6a24-cdf0-4aa3-b571-1d9406b352b5|2021-08-20    |
|18479  |67609108741 |86.4040605836911  |d0e180f0-cb06-42a3-bd1a-c47dca15bc55|2021-08-20    |
|3      |34096466752 |301.5793450525113 |6fb1ff48-24bb-4f97-

In [4]:
sdf_transactions.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- merchant_abn: long (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_datetime: date (nullable = true)



In [23]:
def validateABN(merchant_abn):

    str_abn = str(merchant_abn)
    i = 0
    sum = 0

    for digit in str_abn:

        digit = int(digit)
        
        if i == 0:
            digit -= 1
            sum += digit * 10
        elif i == 1:
            sum += digit
        elif i == 2:
            sum += digit * 3
        elif i == 3:
            sum += digit * 5
        elif i == 4:
            sum += digit * 7
        elif i == 5:
            sum += digit * 9
        elif i == 6:
            sum += digit * 11
        elif i == 7:
            sum += digit * 13
        elif i == 8:
            sum += digit * 15
        elif i == 9:
            sum += digit * 17
        elif i == 10:
            sum += digit * 19

        i += 1
    
    if sum % 89 == 0:
        return True
    else:
        return False

In [24]:
validateABN(51824753556)

True

22/09/08 15:23:54 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 526676 ms exceeds timeout 120000 ms
22/09/08 15:23:54 WARN SparkContext: Killing executors is not supported by current scheduler.
